In [1]:
#pip install importnb

In [2]:
# This notebook executes the training routines of the agent(s) 
import random
import time
from importnb import Notebook
with Notebook():
    import Board
    import Agent
from abc import ABC, abstractmethod

In [3]:
#### Parent Class ####

In [4]:
class Game(ABC):
    def __init__(self, board, agent1):
        """
        clase abstracta Game
        """
        self.board = board
        self.agent1 = agent1

    def reset_game(self):
        self.board.reset()

    @abstractmethod
    def play(self):
        pass # To be implemented by the child classes

In [5]:
#### Child Classes ####

In [6]:
class AgentVsAgent(Game):
    def __init__(self, board, agent1, agent2, epochs = 1):
        super().__init__(board, agent1)
        self.epochs = epochs
        self.agent2 = agent2
        if agent1.chip == agent2.chip:
            raise ValueError("Los agentes deben tener fichas diferentes")

    def play(self):
        """Implementa los juegos entre los dos agentes."""
        
        for epoch in range(self.epochs):
            start = time.time()
            self.reset_game()
            agent_1_turn = random.choice([True, False]) # randomize who gets first turn

            # An EPOCH
            while True :                
                
                if agent_1_turn:# Turno Agente 1
                    self.agent1.play_turn()  
                        
                else: # Turno Agente 2
                    self.agent2.play_turn()

                if self.board.isBoardFull() or self.board.verify_winner(self.agent1.chip) or self.board.verify_winner(self.agent2.chip):
                    break
         
                agent_1_turn = not agent_1_turn
            # END OF AN EPOCH
            
            end = time.time()
            #if epoch%epoch == 0:
            print(f"Epoch # {epoch} took {(end - start):.3f} s")

        self.agent1.export_Qtable('Q_Table_A1')
        self.agent2.export_Qtable('Q_Table_A2')

In [7]:
class PlayerVsAgent(Game):
    def __init__(self, board, agent):
        super().__init__(board, agent)
        self.agent = agent
        #self.player_chip = ?
        if self.agent.chip == "X":
            self.player_chip = "O"
        else:
            self.player_chip = "X"

    def play(self):
        """Implementa un juego entre un humano y un agente."""
                
        play_again = True
        while play_again:
            
            self.reset_game()
            player_turn = random.choice([True, False]) # randomize who gets first turn
            if player_turn:
                print("You start this time")
            else:
                print("I will start this time")
            
            while not (self.board.isBoardFull() or self.board.verify_winner(self.player_chip) or self.board.verify_winner(self.agent.chip)):         

                
                if player_turn:
                    # print the current state of the board
                    self.board.print_board()
                    print("It´s your turn!")

                    # Ask for the next move and place the chip
                    while True:
                        try:
                            user_input = input("What is your move? ")
                            new_col_number = int(user_input)  # Intentar convertir a entero
                            
                            # Verificar que el número esté dentro del rango permitido
                            if not (1 <= new_col_number <= 7):
                                raise ValueError("The column must be a number between 1 and 7.")
                            
                            # Intentar colocar la ficha en el tablero
                            if not self.board.place_chip(new_col_number, self.player_chip):
                                raise ValueError("Invalid move. The column might be full or invalid. Try again.")
                            
                            # Si el movimiento es válido, romper el bucle
                            break
                            
                        except ValueError as e:
                            print(e)

                # FIN DEL TURNO DEL Jugador
                    
                else: # Agents Turn

                    # Verify if the player won on the previous move
                    if self.board.verify_winner(self.player_chip):
                        print("YOU WIN!")
                        # comunicarle al agente que perdió -> asignar la recompensa 
                        break
                    elif self.board.isBoardFull():
                        print("ITS A TIE")
                        # comunicarle al agente que empato -> asignar la recompensa 
                        break
                    
                    print("\nIt´s My turn!")
                    self.agent.play_turn()

                    if self.board.verify_winner(self.agent.chip):
                        print("YOU LOSE!")
                        break
                    # FIN DEL TURNO DEL AGENTE

                player_turn = not player_turn

            # Juego terminó, preguntar si se quiere volver a jugar
            usr_input = ""
            while usr_input.lower() not in ["y", "n", "yes", "no"]:
                usr_input = input("Do you want to play again? (y/n): ").strip()

            if usr_input.lower() in ["y", "yes"]:
                play_again = True  # No es necesario redefinir, pero preferi dejarlo explícito
            elif usr_input.lower() in ["n", "no"]:
                play_again = False

        print("Thanks for playing, Bye!")

In [8]:
### Aux Functions ###

def chooseChips():
        # Choose the chips (player can choose)
    player1_chip = ""
    while player1_chip.lower() not in ["x", "o"]:
        player1_chip = input("Please select your chip (x or o)").strip()
        
    player1_chip = player1_chip.lower()        
    if player1_chip == "x":
        player2_chip = "o"
    else:
        player2_chip = "x"
        
    return player1_chip, player2_chip

def createAgent(): # hay que primero definir como se va a construir el agente para definir esta funcion
    # player1_chip, player2_chip = chooseChips():
    #return Agent(...,player2_chip,...)
    pass


In [9]:
tablero = Board.Connect4Board()
agente = Agent.Agent(tablero, "O")
otroAgente = Agent.Agent(tablero, "X") #TODO: Verificar que los agentes tengan diferente chip
#juego = PlayerVsAgent(tablero,agente)
juego = AgentVsAgent(tablero, agente, otroAgente, epochs = 50)

In [10]:
juego.play()

Epoch # 0 took 0.886 s
Epoch # 1 took 0.546 s
Epoch # 2 took 0.471 s
Epoch # 3 took 0.660 s
Epoch # 4 took 0.663 s
Epoch # 5 took 0.581 s
Epoch # 6 took 0.808 s
Epoch # 7 took 0.234 s
Epoch # 8 took 0.441 s
Epoch # 9 took 1.232 s
Epoch # 10 took 0.665 s
Epoch # 11 took 0.281 s
Epoch # 12 took 0.525 s
Epoch # 13 took 0.512 s
Epoch # 14 took 0.435 s
Epoch # 15 took 0.317 s
Epoch # 16 took 0.329 s
Epoch # 17 took 0.386 s
Epoch # 18 took 0.672 s
Epoch # 19 took 0.261 s
Epoch # 20 took 0.725 s
Epoch # 21 took 0.178 s
Epoch # 22 took 0.775 s
Epoch # 23 took 1.074 s
Epoch # 24 took 0.538 s
Epoch # 25 took 0.998 s
Epoch # 26 took 0.509 s
Epoch # 27 took 0.826 s
Epoch # 28 took 0.274 s
Epoch # 29 took 0.707 s
Epoch # 30 took 0.246 s
Epoch # 31 took 0.219 s
Epoch # 32 took 0.311 s
Epoch # 33 took 0.643 s
Epoch # 34 took 0.411 s
Epoch # 35 took 0.315 s
Epoch # 36 took 0.695 s
Epoch # 37 took 0.692 s
Epoch # 38 took 0.611 s
Epoch # 39 took 0.368 s
Epoch # 40 took 0.558 s
Epoch # 41 took 0.800 s
Ep